# 🍎 SFT LoRA 训练 - 亲属关系任务

使用 M4 芯片进行有监督微调训练，包含：
- 数据加载与增强
- 模型加载（LoRA + float16）
- 训练过程可视化
- 损失曲线绘制
- 模型评估

In [1]:
# 检查运行环境
import torch
import sys

print("=" * 60)
print("🍎 M4 Apple Silicon 训练环境检测")
print("=" * 60)

mps_available = torch.backends.mps.is_available()
mps_built = torch.backends.mps.is_built()

print(f"\n📱 MPS 可用: {mps_available}")
print(f"📱 MPS 已构建: {mps_built}")
print(f"🔵 CUDA 可用: {torch.cuda.is_available()}")
print(f"📦 PyTorch 版本: {torch.__version__}")
print(f"🐍 Python 版本: {sys.version.split()[0]}")

device = torch.device("mps" if mps_available else "cpu")
print(f"\n✅ 使用设备: {device}")

🍎 M4 Apple Silicon 训练环境检测

📱 MPS 可用: True
📱 MPS 已构建: True
🔵 CUDA 可用: False
📦 PyTorch 版本: 2.10.0
🐍 Python 版本: 3.11.14

✅ 使用设备: mps


## 1. 导入依赖

In [2]:
import os
import sys
import json
import random
import logging
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForSeq2Seq,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, TaskType

sys.path.insert(0, str(Path(__file__).parent.parent))

from src.sft import create_lora_config
from src.data.kinship_augment import load_augmented_data

print("✅ 依赖导入完成")

NameError: name '__file__' is not defined

## 2. 配置参数

In [ ]:
# M4 优化配置
config = {
    "model_path": "/Users/xry/.cache/modelscope/hub/models/Qwen/Qwen2___5-7B-Instruct",
    "train_data_path": "./dataset/augmented/train.json",
    "test_data_path": "./dataset/augmented/test.json",
    "output_dir": "./outputs/sft_kinship",
    "epochs": 3,
    "batch_size": 1,
    "learning_rate": 1e-4,
    "max_length": 512,
    "warmup_ratio": 0.1,
    "weight_decay": 0.01,
    "gradient_accumulation_steps": 8,
    "max_grad_norm": 1.0,
    "lora_r": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.1,
    "torch_dtype": torch.float16,
    "device": "mps"
}

os.makedirs(config["output_dir"], exist_ok=True)

print("📋 训练配置:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 3. 加载数据

In [ ]:
# 加载增强后的数据
train_data, test_data = load_augmented_data(
    config["train_data_path"],
    config["test_data_path"]
)

queries = [item['query'] for item in train_data]
answers = [item['answer'] for item in train_data]

test_queries = [item['query'] for item in test_data]
test_answers = [item['answer'] for item in test_data]

print(f"\n📊 数据规模:")
print(f"  训练集: {len(queries)} 条")
print(f"  测试集: {len(test_queries)} 条")

print(f"\n📝 示例数据:")
for i in range(3):
    print(f"  Q: {queries[i]}")
    print(f"  A: {answers[i]}")
    print()

## 4. 加载模型

In [ ]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(
    config["model_path"],
    trust_remote_code=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    config["model_path"],
    torch_dtype=config["torch_dtype"],
    device_map=None,
    trust_remote_code=True
).to(config["device"])

print(f"✅ 模型加载完成")
print(f"  模型参数量: {sum(p.numel() for p in model.parameters()):,}")

## 5. 配置 LoRA

In [ ]:
# 配置 LoRA
lora_config = create_lora_config(
    r=config["lora_r"],
    alpha=config["lora_alpha"],
    dropout=config["lora_dropout"]
)

# 应用 LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 6. 创建数据集

In [ ]:
class SFTDataset(Dataset):
    def __init__(self, queries, answers, tokenizer, max_length=512):
        self.queries = queries
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.queries)
    
    def __getitem__(self, idx):
        query = self.queries[idx]
        answer = self.answers[idx]
        
        conversation = [
            {"role": "system", "content": "You are a helpful assistant specialized in kinship relationships."},
            {"role": "user", "content": query},
            {"role": "assistant", "content": answer}
        ]
        
        try:
            text = self.tokenizer.apply_chat_template(
                conversation,
                tokenize=False,
                add_generation_prompt=False
            )
        except:
            text = f"User: {query}\nAssistant: {answer}"
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# 创建数据集
train_dataset = SFTDataset(queries, answers, tokenizer, config["max_length"])
eval_dataset = SFTDataset(test_queries, test_answers, tokenizer, config["max_length"])

print(f"✅ 数据集创建完成")
print(f"  训练集大小: {len(train_dataset)}")
print(f"  验证集大小: {len(eval_dataset)}")

## 7. 设置优化器和调度器

In [ ]:
# 创建数据加载器
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=data_collator,
    pin_memory=True
)

eval_loader = DataLoader(
    eval_dataset,
    batch_size=config["batch_size"],
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=True
)

# 计算训练步数
num_training_steps = len(train_loader) * config["epochs"] // config["gradient_accumulation_steps"]
num_warmup_steps = int(num_training_steps * config["warmup_ratio"])

# 设置优化器
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    weight_decay=config["weight_decay"]
)

# 设置调度器
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

print(f"✅ 优化器和调度器设置完成")
print(f"  总训练步数: {num_training_steps}")
print(f"  预热步数: {num_warmup_steps}")

## 8. 训练循环（含可视化）

In [ ]:
# 初始化损失记录
train_losses = []
eval_losses = []
learning_rates = []
global_step = 0

# 创建进度条和图表
from tqdm import tqdm

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
plt.ion()  # 开启交互模式

print("🚀 开始训练...")
print("=" * 60)

for epoch in range(config["epochs"]):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']}")
    
    for step, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(config["device"])
        attention_mask = batch['attention_mask'].to(config["device"])
        labels = batch['labels'].to(config["device"])
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss / config["gradient_accumulation_steps"]
        loss.backward()
        
        total_loss += loss.item()
        
        if (step + 1) % config["gradient_accumulation_steps"] == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), config["max_grad_norm"])
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
            global_step += 1
            current_loss = loss.item() * config["gradient_accumulation_steps"]
            train_losses.append(current_loss)
            learning_rates.append(scheduler.get_last_lr()[0])
            
            progress_bar.set_postfix({
                'loss': f'{current_loss:.4f}',
                'lr': f'{scheduler.get_last_lr()[0]:.2e}'
            })
            
            # 实时更新图表
            if global_step % 10 == 0:
                ax1.clear()
                ax1.plot(train_losses, 'b-', alpha=0.6)
                if len(train_losses) > 20:
                    window = min(20, len(train_losses) // 5)
                    smoothed = [sum(train_losses[max(0,i-window):i+1])/(i-max(0,i-window)+1) for i in range(len(train_losses))]
                    ax1.plot(smoothed, 'r-', linewidth=2)
                ax1.set_xlabel('Step')
                ax1.set_ylabel('Loss')
                ax1.set_title('Training Loss')
                ax1.grid(True, alpha=0.3)
                
                ax2.clear()
                ax2.plot(learning_rates, 'g-')
                ax2.set_xlabel('Step')
                ax2.set_ylabel('Learning Rate')
                ax2.set_title('Learning Rate')
                ax2.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.pause(0.01)
    
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Avg Train Loss: {avg_train_loss:.4f}")
    
    # 评估
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids = batch['input_ids'].to(config["device"])
            attention_mask = batch['attention_mask'].to(config["device"])
            labels = batch['labels'].to(config["device"])
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            total_eval_loss += outputs.loss.item()
    
    avg_eval_loss = total_eval_loss / len(eval_loader)
    eval_losses.append(avg_eval_loss)
    print(f"Epoch {epoch+1} - Avg Eval Loss: {avg_eval_loss:.4f}")
    print("-" * 60)

plt.ioff()  # 关闭交互模式
print("✅ 训练完成！")

## 9. 绘制损失曲线

In [ ]:
# 绘制最终损失曲线
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 训练损失
axes[0].plot(train_losses, 'b-', alpha=0.6, label='Raw')
if len(train_losses) > 20:
    window = min(20, len(train_losses) // 5)
    smoothed = [sum(train_losses[max(0,i-window):i+1])/(i-max(0,i-window)+1) for i in range(len(train_losses))]
    axes[0].plot(smoothed, 'r-', linewidth=2, label='Smoothed')
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 评估损失
axes[1].plot(range(1, len(eval_losses)+1), eval_losses, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Evaluation Loss')
axes[1].grid(True, alpha=0.3)

# 学习率
axes[2].plot(learning_rates, 'g-')
axes[2].set_xlabel('Step')
axes[2].set_ylabel('Learning Rate')
axes[2].set_title('Learning Rate Schedule')
axes[2].grid(True, alpha=0.3)

plt.suptitle('SFT Training Results', fontsize=14, y=1.02)
plt.tight_layout()

# 保存图片
loss_plot_path = f"{config['output_dir']}/loss_curve.png"
plt.savefig(loss_plot_path, dpi=150, bbox_inches='tight')
print(f"💾 损失曲线已保存: {loss_plot_path}")

plt.show()

## 10. 保存模型

In [ ]:
# 保存模型
save_path = f"{config['output_dir']}/best_model"
os.makedirs(save_path, exist_ok=True)

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ 模型已保存到: {save_path}")

# 保存训练日志
training_log = {
    "train_losses": train_losses,
    "eval_losses": eval_losses,
    "learning_rates": learning_rates,
    "config": config,
    "final_train_loss": train_losses[-1] if train_losses else None,
    "min_eval_loss": min(eval_losses) if eval_losses else None
}

log_path = f"{config['output_dir']}/training_log.json"
with open(log_path, 'w') as f:
    json.dump(training_log, f, indent=2)

print(f"💾 训练日志已保存: {log_path}")

## 11. 模型评估

In [ ]:
# 测试模型
model.eval()

test_queries_small = test_queries[:10]
test_answers_small = test_answers[:10]

print("🧪 模型测试:")
print("=" * 60)

correct = 0
for query, true_answer in zip(test_queries_small, test_answers_small):
    conversation = [
        {"role": "user", "content": query}
    ]
    
    try:
        prompt = tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=True
        )
    except:
        prompt = f"User: {query}\nAssistant:"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(config["device"])
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    is_correct = response.strip() == true_answer.strip()
    if is_correct:
        correct += 1
    
    print(f"Q: {query}")
    print(f"A: {response.strip()}")
    print(f"✓" if is_correct else "✗" + f" (正确: {true_answer})")
    print("-" * 60)

accuracy = correct / len(test_queries_small) * 100
print(f"\n📊 测试准确率: {accuracy:.1f}% ({correct}/{len(test_queries_small)})")

## 12. 总结

In [ ]:
print("\n" + "=" * 60)
print("📋 训练总结")
print("=" * 60)
print(f"🎯 最终训练损失: {train_losses[-1]:.4f}" if train_losses else "N/A")
print(f"🎯 最低评估损失: {min(eval_losses):.4f}" if eval_losses else "N/A")
print(f"📊 测试准确率: {accuracy:.1f}%")
print(f"💾 模型保存位置: {config['output_dir']}/best_model")
print(f"📈 损失曲线: {config['output_dir']}/loss_curve.png")
print("\n✅ SFT 训练完成！")